1. Write SQL statements to create below objects 
 - A catalog named telecom_catalog_assign </br>
 - A schema landing_zone <br>
 - A volume landing_vol <br>


In [0]:
%sql
create catalog telecom_catalog_assign;


In [0]:
%sql

create schema if not exists telecom_catalog_assign.landing_zone;


In [0]:
%sql
create volume if not exists telecom_catalog_assign.landing_vol;

2. Using dbutils.fs.mkdirs, create below folders<br>
- /Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/ <br>
- /Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/ <br>
- /Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower<br>



In [0]:
dbutils.fs.mkdirs("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
dbutils.fs.mkdirs("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1")
dbutils.fs.mkdirs("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2")

#### Create the customer dataset

In [0]:
# print("h")
customer_csv = ''' 101,Arun,31,Chennai,PREPAID 102,Meera,45,Bangalore,POSTPAID 103,Irfan,29,Hyderabad,PREPAID 104,Raj,52,Mumbai,POSTPAID 105,,27,Delhi,PREPAID 106,Sneha,abc,Pune,PREPAID '''

dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", customer_csv, True)

In [0]:
usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count 101\t320\t1500\t20 102\t120\t4000\t5 103\t540\t600\t52 104\t45\t200\t2 105\t0\t0\t0 '''

In [0]:
# usage_tsv 
dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage",usage_tsv,True)

In [0]:
tower_logs_region = '''event_id|customer_id|tower_id|signal_strength|timestamp 5001|101|TWR01|-80|2025-01-10 10:21:54 5004|104|TWR05|-75|2025-01-10 11:01:12 '''

dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1",tower_logs_region,True)

dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2",tower_logs_region,True)


In [0]:
# dbutils.notebook.run('/Workspace/Users/anto2003.sfn@gmail.com/Usecase_3_solution',60)